In [1]:
import os
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
from sklearn.metrics import accuracy_score
from skimage.feature import hog
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from skimage import io, color, restoration, transform, feature
import random


In [2]:
# Check if GPU is available, else use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cuda')

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [4]:
# Path to the folders containing real and fake images
real_images_path = "/content/gdrive/MyDrive/celebdf/original-cropped-images"
fake_images_path = "/content/gdrive/MyDrive/celebdf/synthetic-cropped-images"


In [5]:
# Function to load and preprocess images
# def load_and_preprocess_images(folder_path):
#     image_list = []
#     count =0
#     for filename in os.listdir(folder_path):
#         image = image.open(os.path.join(folder_path, filename))
#         image = image.resize((64, 64))  # Resize images to a consistent size
#         image = np.array(image)
#         image_list.append(image)
#         count+=1
#         print(f"\rProcessed {count} images", end='')
#     return image_list

def load_and_preprocess_images(folder):
    image_list = []
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    count=0
    for filename in os.listdir(folder):
        if filename.endswith(".jpg"):
            img_path = os.path.join(folder, filename)
            img = io.imread(img_path)
            img_gray = color.rgb2gray(img)
            img_resized = transform.resize(img_gray, (64, 64))  # Resize to a consistent size

            # Convert the NumPy array to a PyTorch tensor
            img_tensor = torch.tensor(img_resized, dtype=torch.float32).to(device)
            image_list.append(img_tensor)

            count+=1
            print(f"\rProcessed {count} images", end='')

    return image_list


In [6]:
# Load and preprocess real and fake images
real_images = load_and_preprocess_images(real_images_path)


Processed 8433 images

In [7]:
fake_images = load_and_preprocess_images(fake_images_path)


Processed 26317 images

In [8]:
# # Create labels (0 for real, 1 for fake)
# real_labels = np.zeros(len(real_images))
# fake_labels = np.ones(len(fake_images))

# Create labels (0 for real, 1 for fake)
real_labels_tensor = torch.zeros(len(real_images), dtype=torch.float32).to(device)
fake_labels_tensor = torch.ones(len(fake_images), dtype=torch.float32).to(device)

real_images_tensor = torch.stack(real_images)
fake_images_tensor = torch.stack(fake_images)

# Combine real and fake data
all_images = torch.cat((real_images_tensor, fake_images_tensor), dim=0)
all_labels = torch.cat((real_labels_tensor, fake_labels_tensor), dim=0)

all_images_numpy = all_images.cpu().numpy()
all_labels_numpy = all_labels.cpu().numpy()


# # Combine real and fake images and labels
# X = np.concatenate((real_images, fake_images))
# y = np.concatenate((real_labels, fake_labels))

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(all_images_numpy, all_labels_numpy, test_size=0.2, random_state=42)


In [9]:
# Convert the NumPy arrays back to PyTorch tensors (and move to GPU if available)
X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.float32).to(device)
y_test = torch.tensor(y_test, dtype=torch.float32).to(device)


In [10]:
# Function to extract HOG features from a list of images
def extract_hog_features(images):
    hog_features = []
    for image in images:
        hog_feature = hog(image, pixels_per_cell=(8, 8), cells_per_block=(2, 2))
        hog_features.append(hog_feature)
    return np.array(hog_features)


In [11]:
from skimage.feature import hog
from sklearn.preprocessing import StandardScaler

# Move tensors to CPU
X_train_cpu = X_train.cpu().numpy()
X_test_cpu = X_test.cpu().numpy()

# Extract HOG features from training and testing data
X_train_hog = extract_hog_features(X_train_cpu)
X_test_hog = extract_hog_features(X_test_cpu)

# Standardize the HOG features
scaler = StandardScaler()
X_train_hog = scaler.fit_transform(X_train_hog)
X_test_hog = scaler.transform(X_test_hog)


In [12]:
# Convert HOG features to PyTorch tensors
X_train_hog_tensor = torch.tensor(X_train_hog, dtype=torch.float32)
X_test_hog_tensor = torch.tensor(X_test_hog, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Move tensors to CPU
X_train_hog_tensor = X_train_hog_tensor.cpu()
X_test_hog_tensor = X_test_hog_tensor.cpu()
y_train_tensor = y_train_tensor.cpu()
y_test_tensor = y_test_tensor.cpu()

# Convert to NumPy arrays
X_train_hog = X_train_hog_tensor.numpy()
X_test_hog = X_test_hog_tensor.numpy()
y_train = y_train_tensor.numpy()
y_test = y_test_tensor.numpy()


<ipython-input-12-5b5ca016d14e>:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
<ipython-input-12-5b5ca016d14e>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_test_tensor = torch.tensor(y_test, dtype=torch.float32)


In [13]:
# Create a simple neural network
class NeuralNetwork(torch.nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.fc1 = torch.nn.Linear(X_train_hog.shape[1], 128)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(128, 1)
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)
        return x

# Initialize the model and define loss function and optimizer
model = NeuralNetwork()
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)



In [14]:
# Train the model
num_epochs = 10
for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = model(torch.tensor(X_train_hog, dtype=torch.float32))
    loss = criterion(outputs, torch.tensor(y_train, dtype=torch.float32).view(-1, 1))
    loss.backward()
    optimizer.step()
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")


Epoch [1/10], Loss: 0.7184
Epoch [2/10], Loss: 0.5977
Epoch [3/10], Loss: 0.5246
Epoch [4/10], Loss: 0.4726
Epoch [5/10], Loss: 0.4375
Epoch [6/10], Loss: 0.4170
Epoch [7/10], Loss: 0.4064
Epoch [8/10], Loss: 0.4005
Epoch [9/10], Loss: 0.3952
Epoch [10/10], Loss: 0.3888


In [15]:
# Evaluate the model on the test set
with torch.no_grad():
    outputs = model(torch.tensor(X_test_hog, dtype=torch.float32))
    predicted = (outputs >= 0.5).squeeze().cpu().numpy()

# Calculate accuracy
accuracy = accuracy_score(y_test, predicted)
print(f"Accuracy: {accuracy * 100:.2f}%")


Accuracy: 82.22%


In [17]:
pip install torchviz


  Preparing metadata (setup.py) ... done
  Created wheel for torchviz: filename=torchviz-0.0.2-py3-none-any.whl size=4130 sha256=6c823014ee04bf4438723de28f19f4e5737ae83df927656b6b0c0bc65d1d3e42
  Stored in directory: /root/.cache/pip/wheels/4c/97/88/a02973217949e0db0c9f4346d154085f4725f99c4f15a87094
Successfully built torchviz


In [19]:
import torchviz

# Create a sample input tensor with the correct shape
input_tensor = torch.randn(1, 1764)

# Pass the input tensor to the model to generate the architecture
dot = torchviz.make_dot(model(input_tensor))

# Render and save the model architecture
dot.render("model_architecture.png", format="png")



'model_architecture.png.png'